In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.preprocessing import MinMaxScaler

### Đọc dữ liệu từ file

In [2]:
EC = pd.read_csv('/kaggle/input/ec40kdata/EC.csv')
EC.drop(columns=['Time (seconds)'], inplace=True)                    
EC

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16
0,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,55139.95,50669.50,9626.26,9762.62,24544.02,21420.68,7650.61,6928.42
1,0.0,0.0,-37.78,33.48,-29.05,6.03,-2.44,-25.33,-3.09,-4.56,10640.31,12029.40,4348.10,4437.45,7385.40,6829.65,3170.98,2872.14
2,0.0,0.0,46.27,257.27,-15.76,39.05,24.96,23.33,8.31,17.94,2984.76,3880.04,2516.30,2586.04,3578.80,3457.75,1754.13,1582.84
3,0.0,0.0,304.06,733.38,32.17,46.11,145.48,233.52,43.65,57.46,1531.53,2028.66,1687.94,1733.92,2235.14,2237.82,1143.56,1031.91
4,0.0,0.0,609.38,1362.42,58.61,121.74,369.97,612.68,89.50,122.74,1049.37,1414.35,1291.00,1330.15,1652.25,1705.09,839.04,754.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42078,0.0,0.0,1141.54,106.76,2110.46,2253.45,1201.66,1617.22,2088.30,2265.19,1069.47,1070.47,2574.45,2059.35,715.97,777.62,2510.44,1856.65
42079,0.0,0.0,1138.71,117.91,2107.36,2251.73,1200.44,1601.21,2091.16,2295.13,1072.86,1082.05,2566.47,2072.39,709.62,776.49,2513.24,1857.34
42080,0.0,0.0,1129.02,110.42,2115.00,2249.29,1210.45,1611.95,2087.11,2264.94,1073.66,1067.08,2565.70,2061.96,707.38,797.51,2523.95,1847.05
42081,0.0,0.0,1144.77,102.11,2108.55,2248.32,1211.88,1614.37,2114.28,2269.11,1067.28,1067.08,2578.82,2080.70,698.61,774.60,2519.36,1854.37


### Lọc bỏ các mixture chứa cả cặp khí bằng 0

In [3]:
EC1 = EC.loc[~((EC["CO2 conc (ppm)"] == 0) & (EC["Ethylene conc (ppm)"] == 0))] 
EC1 = EC1.copy()
EC1['Index'] = range(1, len(EC1) + 1)
EC1.set_index('Index', inplace=True)
EC1

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16
Index,,,,,,,,,,,,,,,,,,
1,533.33,0.0,1465.68,2178.15,1130.64,1298.89,1437.51,2047.77,1414.13,1455.21,393.65,501.85,1484.97,1433.68,1015.29,979.47,1505.83,1318.04
2,533.33,0.0,1465.68,2174.77,1137.90,1297.65,1419.01,2047.53,1425.81,1460.33,381.89,485.54,1478.10,1463.54,1019.24,980.64,1493.56,1310.96
3,533.33,0.0,1454.99,2167.54,1141.74,1285.61,1439.21,2040.21,1420.71,1454.57,383.46,488.04,1476.39,1464.61,999.32,970.45,1495.50,1326.81
4,533.33,0.0,1464.40,2164.65,1129.02,1291.00,1423.05,2040.92,1420.50,1446.24,363.50,494.13,1471.68,1454.78,1001.49,983.00,1526.99,1314.50
5,533.33,0.0,1449.44,2149.74,1133.86,1296.40,1426.45,2038.33,1414.35,1453.50,391.72,475.52,1458.20,1457.77,1006.22,981.43,1498.08,1324.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,533.33,0.0,2985.86,115.08,4800.90,5378.07,2112.85,2505.34,5437.96,5795.97,1795.65,1803.58,5686.27,4683.46,943.09,1057.72,5717.17,5313.29
29580,533.33,0.0,2994.65,115.08,4801.25,5387.70,2096.16,2503.31,5453.10,5802.06,1806.53,1816.07,5673.07,4666.28,932.97,1049.96,5720.39,5352.71
29581,533.33,0.0,2998.22,119.24,4807.32,5371.91,2093.06,2495.17,5469.06,5797.19,1792.03,1825.16,5680.27,4623.35,928.69,1058.51,5725.22,5369.22


### Gán nhãn cho dữ liệu
Quy tắc gán nhãn là cứ một lần thay đổi mixture là 1 nhãn, bắt đầu từ 1.

In [4]:
def label_groups(df):
    # Create a new column 'Labels' initialized with 0
    df['Labels'] = 0
    # Initialize the group label counter
    group_label = 1
    # Get the initial mixture values
    prev_mixture = (df['CO2 conc (ppm)'].iloc[0], df['Ethylene conc (ppm)'].iloc[0])
    # Iterate over the rows in the DataFrame
    for i in range(1, len(df)):
        current_mixture = (df['CO2 conc (ppm)'].iloc[i], df['Ethylene conc (ppm)'].iloc[i])
        # Check if the mixture values have changed
        if current_mixture != prev_mixture:
            group_label += 1
            prev_mixture = current_mixture
        # Assign the group label
        df.at[i, 'Labels'] = group_label
    return df

In [5]:
EC2 = label_groups(EC1)
EC2.loc[EC2.index[-1], 'Labels'] = 277
EC2

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Labels
Index,,,,,,,,,,,,,,,,,,,
1,533.33,0.0,1465.68,2178.15,1130.64,1298.89,1437.51,2047.77,1414.13,1455.21,393.65,501.85,1484.97,1433.68,1015.29,979.47,1505.83,1318.04,1
2,533.33,0.0,1465.68,2174.77,1137.90,1297.65,1419.01,2047.53,1425.81,1460.33,381.89,485.54,1478.10,1463.54,1019.24,980.64,1493.56,1310.96,1
3,533.33,0.0,1454.99,2167.54,1141.74,1285.61,1439.21,2040.21,1420.71,1454.57,383.46,488.04,1476.39,1464.61,999.32,970.45,1495.50,1326.81,1
4,533.33,0.0,1464.40,2164.65,1129.02,1291.00,1423.05,2040.92,1420.50,1446.24,363.50,494.13,1471.68,1454.78,1001.49,983.00,1526.99,1314.50,1
5,533.33,0.0,1449.44,2149.74,1133.86,1296.40,1426.45,2038.33,1414.35,1453.50,391.72,475.52,1458.20,1457.77,1006.22,981.43,1498.08,1324.93,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,533.33,0.0,2985.86,115.08,4800.90,5378.07,2112.85,2505.34,5437.96,5795.97,1795.65,1803.58,5686.27,4683.46,943.09,1057.72,5717.17,5313.29,277
29580,533.33,0.0,2994.65,115.08,4801.25,5387.70,2096.16,2503.31,5453.10,5802.06,1806.53,1816.07,5673.07,4666.28,932.97,1049.96,5720.39,5352.71,277
29581,533.33,0.0,2998.22,119.24,4807.32,5371.91,2093.06,2495.17,5469.06,5797.19,1792.03,1825.16,5680.27,4623.35,928.69,1058.51,5725.22,5369.22,277


### Kiểm tra các nhóm có ít hơn 64 dòng

In [6]:
def check_small_groups(df):
    # Group the DataFrame by the 'Labels' column
    grouped = df.groupby('Labels')
    # Initialize a list to store labels of small groups
    small_groups = []
    # Iterate over each group
    for label, group in grouped:
        # Check if the group has fewer than 64 rows
        if len(group) < 64:
            small_groups.append((label, len(group)))
    # Print out the labels and sizes of the small groups
    if small_groups:
        print(f"There are {len(small_groups)} groups with fewer than 64 rows:")
        for label, size in small_groups:
            print(f"Label {label}: {size} rows")
    else:
        print("All groups have 64 or more rows.")
    return small_groups


In [7]:
# Example usage:
small_groups = check_small_groups(EC2)

There are 32 groups with fewer than 64 rows:
Label 17: 44 rows
Label 18: 37 rows
Label 26: 38 rows
Label 34: 7 rows
Label 38: 42 rows
Label 47: 58 rows
Label 48: 38 rows
Label 54: 10 rows
Label 56: 4 rows
Label 58: 23 rows
Label 66: 8 rows
Label 76: 21 rows
Label 87: 46 rows
Label 94: 57 rows
Label 95: 55 rows
Label 138: 44 rows
Label 139: 45 rows
Label 148: 10 rows
Label 155: 60 rows
Label 156: 31 rows
Label 181: 37 rows
Label 211: 49 rows
Label 212: 49 rows
Label 213: 32 rows
Label 231: 20 rows
Label 240: 63 rows
Label 241: 20 rows
Label 242: 35 rows
Label 243: 50 rows
Label 244: 58 rows
Label 245: 34 rows
Label 251: 22 rows


### Loại bỏ các nhãn có ít hơn 64 dòng

In [8]:
def drop_small_groups(df):
    # Group the DataFrame by the 'Labels' column
    grouped = df.groupby('Labels')
    # Initialize a list to store labels of small groups
    labels_to_drop = []  
    # Iterate over each group
    for label, group in grouped:
        # Check if the group has fewer than 64 rows
        if len(group) < 64:
            labels_to_drop.append(label)
    # Drop the rows with labels in labels_to_drop
    df = df[~df['Labels'].isin(labels_to_drop)].reset_index(drop=True)
    return df

In [9]:
EC3 = drop_small_groups(EC2)
EC3 = EC3.reset_index(drop=True)
EC3

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Labels
0,533.33,0.0,1465.68,2178.15,1130.64,1298.89,1437.51,2047.77,1414.13,1455.21,393.65,501.85,1484.97,1433.68,1015.29,979.47,1505.83,1318.04,1
1,533.33,0.0,1465.68,2174.77,1137.90,1297.65,1419.01,2047.53,1425.81,1460.33,381.89,485.54,1478.10,1463.54,1019.24,980.64,1493.56,1310.96,1
2,533.33,0.0,1454.99,2167.54,1141.74,1285.61,1439.21,2040.21,1420.71,1454.57,383.46,488.04,1476.39,1464.61,999.32,970.45,1495.50,1326.81,1
3,533.33,0.0,1464.40,2164.65,1129.02,1291.00,1423.05,2040.92,1420.50,1446.24,363.50,494.13,1471.68,1454.78,1001.49,983.00,1526.99,1314.50,1
4,533.33,0.0,1449.44,2149.74,1133.86,1296.40,1426.45,2038.33,1414.35,1453.50,391.72,475.52,1458.20,1457.77,1006.22,981.43,1498.08,1324.93,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28431,533.33,0.0,2985.86,115.08,4800.90,5378.07,2112.85,2505.34,5437.96,5795.97,1795.65,1803.58,5686.27,4683.46,943.09,1057.72,5717.17,5313.29,277
28432,533.33,0.0,2994.65,115.08,4801.25,5387.70,2096.16,2503.31,5453.10,5802.06,1806.53,1816.07,5673.07,4666.28,932.97,1049.96,5720.39,5352.71,277
28433,533.33,0.0,2998.22,119.24,4807.32,5371.91,2093.06,2495.17,5469.06,5797.19,1792.03,1825.16,5680.27,4623.35,928.69,1058.51,5725.22,5369.22,277
28434,533.33,0.0,3008.68,108.09,4803.75,5369.22,2104.50,2507.38,5444.17,5820.37,1807.89,1828.35,5665.08,4642.17,944.65,1061.70,5734.88,5361.54,277


### Reset lại tên từng nhóm

In [10]:
def reset_labels(df):    
    # Get unique labels sorted by their original order
    unique_labels = sorted(df['Labels'].unique())
    # Create a mapping from the old labels to new sequential labels
    label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels, start=1)}
    # Apply the mapping to the 'Labels' column
    df['Labels'] = df['Labels'].map(label_mapping)
    return df

In [11]:
EC4 = reset_labels(EC3)
EC4

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Labels
0,533.33,0.0,1465.68,2178.15,1130.64,1298.89,1437.51,2047.77,1414.13,1455.21,393.65,501.85,1484.97,1433.68,1015.29,979.47,1505.83,1318.04,1
1,533.33,0.0,1465.68,2174.77,1137.90,1297.65,1419.01,2047.53,1425.81,1460.33,381.89,485.54,1478.10,1463.54,1019.24,980.64,1493.56,1310.96,1
2,533.33,0.0,1454.99,2167.54,1141.74,1285.61,1439.21,2040.21,1420.71,1454.57,383.46,488.04,1476.39,1464.61,999.32,970.45,1495.50,1326.81,1
3,533.33,0.0,1464.40,2164.65,1129.02,1291.00,1423.05,2040.92,1420.50,1446.24,363.50,494.13,1471.68,1454.78,1001.49,983.00,1526.99,1314.50,1
4,533.33,0.0,1449.44,2149.74,1133.86,1296.40,1426.45,2038.33,1414.35,1453.50,391.72,475.52,1458.20,1457.77,1006.22,981.43,1498.08,1324.93,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28431,533.33,0.0,2985.86,115.08,4800.90,5378.07,2112.85,2505.34,5437.96,5795.97,1795.65,1803.58,5686.27,4683.46,943.09,1057.72,5717.17,5313.29,245
28432,533.33,0.0,2994.65,115.08,4801.25,5387.70,2096.16,2503.31,5453.10,5802.06,1806.53,1816.07,5673.07,4666.28,932.97,1049.96,5720.39,5352.71,245
28433,533.33,0.0,2998.22,119.24,4807.32,5371.91,2093.06,2495.17,5469.06,5797.19,1792.03,1825.16,5680.27,4623.35,928.69,1058.51,5725.22,5369.22,245
28434,533.33,0.0,3008.68,108.09,4803.75,5369.22,2104.50,2507.38,5444.17,5820.37,1807.89,1828.35,5665.08,4642.17,944.65,1061.70,5734.88,5361.54,245


### Chỉ giữ lại duy nhất 64 dòng đầu tiên của mỗi nhóm

In [12]:
def get64rows(df):
    # Initialize an empty list to store the selected rows
    selected_rows = []
    # Group the DataFrame by the 'Labels' column
    grouped = df.groupby('Labels')
    # Iterate over each group
    for _, group in grouped:
        # Select the first 64 rows from each group
        selected_rows.append(group.iloc[:64])
    # Concatenate the selected rows into a new DataFrame and reset index
    df = pd.concat(selected_rows).reset_index(drop=True)
    return df

In [13]:
EC5=get64rows(EC4)
EC5

,CO2 conc (ppm),Ethylene conc (ppm),Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Labels
0,533.33,0.0,1465.68,2178.15,1130.64,1298.89,1437.51,2047.77,1414.13,1455.21,393.65,501.85,1484.97,1433.68,1015.29,979.47,1505.83,1318.04,1
1,533.33,0.0,1465.68,2174.77,1137.90,1297.65,1419.01,2047.53,1425.81,1460.33,381.89,485.54,1478.10,1463.54,1019.24,980.64,1493.56,1310.96,1
2,533.33,0.0,1454.99,2167.54,1141.74,1285.61,1439.21,2040.21,1420.71,1454.57,383.46,488.04,1476.39,1464.61,999.32,970.45,1495.50,1326.81,1
3,533.33,0.0,1464.40,2164.65,1129.02,1291.00,1423.05,2040.92,1420.50,1446.24,363.50,494.13,1471.68,1454.78,1001.49,983.00,1526.99,1314.50,1
4,533.33,0.0,1449.44,2149.74,1133.86,1296.40,1426.45,2038.33,1414.35,1453.50,391.72,475.52,1458.20,1457.77,1006.22,981.43,1498.08,1324.93,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15675,533.33,0.0,2529.31,114.41,4727.81,5246.41,2070.49,2504.83,5454.27,5773.26,1618.98,1637.02,5552.07,4560.62,935.30,1040.03,5633.19,5282.44,245
15676,533.33,0.0,2550.30,127.75,4754.69,5279.02,2054.62,2473.10,5452.72,5771.23,1633.06,1631.74,5515.94,4568.22,936.47,1058.51,5663.08,5266.11,245
15677,533.33,0.0,2566.22,124.41,4766.39,5294.62,2057.70,2471.08,5455.05,5777.31,1642.76,1646.07,5537.52,4587.93,933.75,1064.09,5664.68,5258.15,245
15678,533.33,0.0,2591.97,109.75,4740.88,5266.87,2070.25,2494.66,5466.72,5748.60,1646.95,1660.21,5563.10,4568.22,921.31,1062.10,5657.89,5308.33,245


### Chuyển mỗi nhóm thành 1 ảnh
Chỉ chuyển dữ liệu của các Sensor với quy luật 32 dòng đầu bên trái và 32 dòng sau bên phải để tạo thành ảnh vuông 32x32

In [14]:
def normalize(data):
    scaler = MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [ ]:
def process64rows(data):
    img_w = 64  # Set width to 64
    img_h = 64  # Set height to 64
    # Normalize the sensor data (excluding the mixture columns)
    sensor_columns = [col for col in data.columns if 'Sensor' in col]
    data[sensor_columns] = normalize(data[sensor_columns])
    
    # Ensure 'imgs' directory exists
    if not os.path.exists('imgs'):
        os.makedirs('imgs')

    labels = []
    cols_order = sensor_columns  # Use sensor columns for image data

    grouped = data.groupby('Labels')

    for group, group_data in grouped:
        # Initialize a blank white image of 64x64
        img = np.full((img_h, img_w), 255, dtype=np.uint8)
        
        # Get CO2 and Ethylene values for the group
        co2_label = group_data.iloc[0]["CO2 conc (ppm)"]
        ethylene_label = group_data.iloc[0]["Ethylene conc (ppm)"]
        
        # Create the image name
        img_name = f'{group}.png'
        labels.append([img_name, co2_label, ethylene_label])

        # Fill the entire width of the image by repeating each column 4 times
        for i in range(16):  # Iterate over 16 sensor columns
            for j in range(64):  # Ensure index does not exceed available data
                value = group_data.iloc[j][cols_order[i]]
                for k in range(4):  # Repeat each column 4 times
                    img[j][i * 4 + k] = int(255 * value)

        # Save the image
        image = Image.fromarray(img)
        image.save(f'imgs/{img_name}')

    # Save labels to CSV
    labels_df = pd.DataFrame(labels, columns=['Image', 'CO2 conc (ppm)', 'Ethylene conc (ppm)'])
    labels_df.to_csv('image_labels.csv', index=False)

In [16]:
process64rows(EC5)

In [17]:
mothaiEC = pd.read_csv('/kaggle/working/image_labels.csv')
mothaiEC

,Image,CO2 conc (ppm),Ethylene conc (ppm)
0,1.png,533.33,0.00
1,2.png,533.33,0.00
2,3.png,0.00,16.67
3,4.png,0.00,10.00
4,5.png,0.00,6.67
...,...,...,...
240,241.png,0.00,16.67
241,242.png,0.00,10.00
242,243.png,0.00,6.67
243,244.png,200.00,0.00
